# Write a master CSV file with data for neural network/machine learning 

Import necessary libraries

In [1]:
import os
import glob
import pandas as pd

In [7]:
cwd = os.getcwd()
parent = os.path.dirname(cwd)
parent

'/home/niko/Documents/Python/reynoldsDeepL'

In [10]:
case_dir = os.path.join(parent, "flowCases/")
case_list = os.listdir(case_dir)
case_list[:5]

['channelStripRe=532.38',
 'channelStripRe=760.11',
 'leftStripBendRe=1898.78',
 'rightStripBendRe=886.63',
 'forwardStepRe=1468.62']

Read a value from a sample case

In [12]:
case_path = os.path.join(case_dir, case_list[0])
postFile = glob.glob(f"{case_path}/postProcessing/*/0/surfaceFieldValue.dat")[0]
postFile

'/home/niko/Documents/Python/reynoldsDeepL/flowCases/channelStripRe=532.38/postProcessing/patchAverage(name=inlet,p)/0/surfaceFieldValue.dat'

In [13]:
with open(postFile) as f:
    lines = f.readlines()
    print(float(lines[-1].split()[1]))  # last time, second column

11.9224


Let's turn what we did into a script for all of the cases and create the master CSV file

In [14]:
# Function to read value from file
def read_value(path):
        try:
            with open(path) as f:
                lines = f.readlines()
                return float(lines[-1].split()[1])  # last time, second column
        except:
            return None

In [18]:
import re
pattern = r'\d+\.\d{2}'

In [19]:
cwd = os.getcwd()
parent = os.path.dirname(cwd)
base_dir = os.path.join(parent, "flowCases/")
rows = []

for case in os.listdir(base_dir):
    case_dir = os.path.join(base_dir, case)
    if not os.path.isdir(case_dir):
        continue
    p_in = read_value(glob.glob(f"{case_dir}/postProcessing/*/0/surfaceFieldValue.dat")[0])
    # Create dictionary for each case
    rows.append({
        "p_in": p_in,
        "Re": re.search(pattern, case).group(0) if re.search(pattern, case) else None,
        "Geometry": case.split('=')[0] if '=' in case else None        
    })

In [20]:
df = pd.DataFrame(rows)
df.head(3)

,p_in,Re,Geometry
0,11.922400,532.38,channelStripRe
1,12.093420,760.11,channelStripRe
2,6.648019,1898.78,leftStripBendRe


In [22]:
csv_folder = os.path.join(parent, "data")
if not os.path.exists(csv_folder):
    os.makedirs(csv_folder)
csv_path = os.path.join(csv_folder, "master_results.csv")
df.to_csv(csv_path, index=False)